In [369]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [370]:
cd ~/demres

/Users/zurfarosa/demres


In [371]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.common import codelists
from demres.common.helper_functions import *
from demins.functions import *

In [372]:
pd.set_option('display.max_columns', None)

In [373]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)

In [374]:
pt_features['gender']-=1

In [375]:
pt_features['yob'] = pt_features['yob'].astype(str).str[2:]

In [376]:
pt_features.columns

Index(['patid', 'gender', 'yob', 'pracid', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'insom_count_window0', 'insom_count_window1', 'stroke_window0',
       'stroke_window1',
       'CHD_heart_failure_and_peripheral_vascular_disease_window0',
       'CHD_heart_failure_and_peripheral_vascular_disease_window1',
       'hypertension_window0', 'hypertension_window1', 'diabetes_window0',
       'diabetes_window1', 'clin_sig_alcohol_use_window0',
       'clin_sig_alcohol_use_window1', 'mental_illness_non_smi_window0',
       'mental_illness_non_smi_window1', 'mental_illness_smi_window0',
       'mental_illness_smi_window1', 'sleep_apnoea_window0',
       'sleep_apnoea_window1', 'intellectual_disability_window0',
       'intellectual_disability_window1', 'current_smoker_window0',
       'current_smoker_window1', 'asthma_window0', 'asthma_window1',
       'COPD_window0', 'COPD_window1', 'CKD_window0', 'CKD_window1',
       'mood_stabili

In [377]:
pt_features['isCase'] = pt_features['isCase'].astype(int)

In [378]:
pt_features['intercept'] = 1.0

In [379]:
pt_features.sample(5)

,patid,gender,yob,pracid,index_date,isCase,final dementia medcode,data_start,data_end,matchid,insom_count_window0,insom_count_window1,stroke_window0,stroke_window1,CHD_heart_failure_and_peripheral_vascular_disease_window0,CHD_heart_failure_and_peripheral_vascular_disease_window1,hypertension_window0,hypertension_window1,diabetes_window0,diabetes_window1,clin_sig_alcohol_use_window0,clin_sig_alcohol_use_window1,mental_illness_non_smi_window0,mental_illness_non_smi_window1,mental_illness_smi_window0,mental_illness_smi_window1,sleep_apnoea_window0,sleep_apnoea_window1,intellectual_disability_window0,intellectual_disability_window1,current_smoker_window0,current_smoker_window1,asthma_window0,asthma_window1,COPD_window0,COPD_window1,CKD_window0,CKD_window1,mood_stabilisers_pdds_window0,mood_stabilisers_pdds_window1,sedatives_pdds_window0,sedatives_pdds_window1,fgas_pdds_window0,fgas_pdds_window1,sgas_pdds_window0,sgas_pdds_window1,sga_depots_pdds_window0,sga_depots_pdds_window1,fga_depots_pdds_window0,fga_depots_pdds_window1,antidepressants_pdds_window0,antidepressants_pdds_window1,consultation_count_window0,consultation_count_window1,intercept
9622,9510164,0,24,164,2008-05-26,1,1917.0,1996-05-13,2010-09-30,24299.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,49,142,1.0
3152,5249183,1,36,183,2012-02-02,1,8195.0,2000-09-22,2013-04-15,27834.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,40.55,0.00,0.0,55.97,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,152,212,1.0
3742,2825005,1,31,5,2009-06-23,1,26270.0,1997-11-21,2010-09-02,1269.0,1,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.93,23.35,99.78,0.0,83.96,0.0,0.0,0.0,0.0,0.0,0.0,0.00,83.13,59,423,1.0
7792,18430306,1,26,306,2010-09-26,0,0.0,1999-08-06,2013-05-10,46357.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2.31,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,184,173,1.0
17607,2416321,1,34,321,2011-03-15,0,0.0,1996-04-04,2013-05-13,48964.0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,33.68,600.47,213,355,1.0


In [399]:
training_cols_window0 = [col for col in pt_features.columns if all([col not in ['patid','index_date','matchid','data_start','data_end','final dementia medcode','isCase','yob','pracid'],'window1' not in col])]
training_cols_window0

['gender',
 'insom_count_window0',
 'stroke_window0',
 'CHD_heart_failure_and_peripheral_vascular_disease_window0',
 'hypertension_window0',
 'diabetes_window0',
 'clin_sig_alcohol_use_window0',
 'mental_illness_non_smi_window0',
 'mental_illness_smi_window0',
 'sleep_apnoea_window0',
 'intellectual_disability_window0',
 'current_smoker_window0',
 'asthma_window0',
 'COPD_window0',
 'CKD_window0',
 'mood_stabilisers_pdds_window0',
 'sedatives_pdds_window0',
 'fgas_pdds_window0',
 'sgas_pdds_window0',
 'sga_depots_pdds_window0',
 'fga_depots_pdds_window0',
 'antidepressants_pdds_window0',
 'consultation_count_window0',
 'intercept']

In [400]:
# training_cols = [
#     'insom_count_window0',
#     'sedatives_pdds_window0',
#     'gender',
#     'CHD_heart_failure_and_peripheral_vascular_disease_window0',
#     'hypertension_window0', 
#     'diabetes_window0',
#     'stroke_window0',
# #     'mental_illness_smi_window0',
# #     'mental_illness_non_smi_window0',
#     'sleep_apnoea_window0',
#     'current_smoker_window0',
#     'asthma_window0',
#     'COPD_window0',
#     'CKD_window0',
# #     'mood_stabilisers_pdds_window0',
# #     'fgas_pdds_window0',
# #     'sgas_pdds_window0', 
# #     'fga_depots_pdds_window0',
# #     'antidepressants_pdds_window0',
#      'consultation_count_window0'
# #     'intellectual_disability_window0',
# #     'sga_depots_pdds_window0'
# ]

In [401]:
univariate_results, multivariate_results = get_univariate_and_multivariate_results(pt_features,training_cols_window0)

Optimization terminated successfully.
         Current function value: 0.686467
         Iterations 6


In [402]:
univariate_results

,odds_ratio,p_value
gender,1.0000,1.000
insom_count_window0,1.0570,0.002
stroke_window0,1.2732,0.000
CHD_heart_failure_and_peripheral_vascular_disease_window0,1.0944,0.015
hypertension_window0,0.9582,0.144
diabetes_window0,1.1879,0.002
mental_illness_non_smi_window0,1.4321,0.000
mental_illness_smi_window0,1.8070,0.000
current_smoker_window0,1.0243,0.555
asthma_window0,1.0436,0.536


In [403]:
multivariate_results

,odds_ratio,p_value
gender,0.9723,0.363
insom_count_window0,1.0186,0.304
stroke_window0,1.1721,0.020
CHD_heart_failure_and_peripheral_vascular_disease_window0,0.9937,0.882
hypertension_window0,0.8954,0.001
diabetes_window0,1.0634,0.299
mental_illness_non_smi_window0,1.3541,0.000
mental_illness_smi_window0,1.2833,0.016
current_smoker_window0,0.9841,0.718
asthma_window0,0.9570,0.674


In [404]:
training_cols_window1 = [col for col in pt_features.columns if all([col not in ['patid','index_date','matchid','data_start','data_end','final dementia medcode','isCase','yob','pracid'],'window0' not in col])]
training_cols_window1

['gender',
 'insom_count_window1',
 'stroke_window1',
 'CHD_heart_failure_and_peripheral_vascular_disease_window1',
 'hypertension_window1',
 'diabetes_window1',
 'clin_sig_alcohol_use_window1',
 'mental_illness_non_smi_window1',
 'mental_illness_smi_window1',
 'sleep_apnoea_window1',
 'intellectual_disability_window1',
 'current_smoker_window1',
 'asthma_window1',
 'COPD_window1',
 'CKD_window1',
 'mood_stabilisers_pdds_window1',
 'sedatives_pdds_window1',
 'fgas_pdds_window1',
 'sgas_pdds_window1',
 'sga_depots_pdds_window1',
 'fga_depots_pdds_window1',
 'antidepressants_pdds_window1',
 'consultation_count_window1',
 'intercept']

In [405]:
univariate_results, multivariate_results = get_univariate_and_multivariate_results(pt_features,training_cols_window1)

Optimization terminated successfully.
         Current function value: 0.663123
         Iterations 5


In [406]:
univariate_results

,odds_ratio,p_value
gender,1.0000,1.000
insom_count_window1,1.0435,0.004
stroke_window1,1.8379,0.000
CHD_heart_failure_and_peripheral_vascular_disease_window1,0.9690,0.433
hypertension_window1,0.9526,0.161
diabetes_window1,1.1198,0.029
mental_illness_non_smi_window1,2.1245,0.000
mental_illness_smi_window1,2.9423,0.000
current_smoker_window1,1.0655,0.107
asthma_window1,0.9676,0.700


In [407]:
multivariate_results

,odds_ratio,p_value
gender,0.9454,0.074
insom_count_window1,0.9919,0.564
stroke_window1,1.6888,0.000
CHD_heart_failure_and_peripheral_vascular_disease_window1,0.7574,0.000
hypertension_window1,0.9051,0.012
diabetes_window1,0.9290,0.198
mental_illness_non_smi_window1,2.1059,0.000
mental_illness_smi_window1,1.6698,0.000
current_smoker_window1,1.0747,0.103
asthma_window1,1.1387,0.237


In [ ]:
ss